<a href="https://colab.research.google.com/github/park-geun-hyeong/Kaggle_project/blob/main/Classification/Binary_Classification/Porto/make_data_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

import os
from glob import glob

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.feature_selection import SelectFromModel, VarianceThreshold

In [3]:
path = '/content/drive/MyDrive/Kaggle/Classification/kaggle_Porto/' 

train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')  

In [5]:
print(train.shape, test.shape, submission.shape) ## There are 58 variables

(595212, 59) (892816, 58) (892816, 2)


In [6]:
train.isna().sum().sum() ## No missing values

0

## Make dataframe according to features type

In [7]:
train

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.5,0.3,0.692820,10,1,-1,0,1,1,1,1,0,1,31,3,0.374166,0.684631,0.385487,2.645751,0.4,0.5,0.3,3,0,9,0,9,1,12,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,0.7,1.382027,9,1,-1,0,-1,15,0,0,2,1,63,2,0.387298,0.972145,-1.000000,3.605551,0.2,0.2,0.0,2,4,8,6,8,2,12,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,0,0,0,0,0,0,12,1,0,0,0.9,0.2,0.659071,7,1,-1,0,-1,1,1,1,2,1,31,3,0.397492,0.596373,0.398748,1.732051,0.4,0.0,0.3,3,2,7,4,8,0,10,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.9,0.4,0.698212,11,1,-1,0,-1,11,1,1,2,1,101,3,0.374166,0.764434,0.384968,3.162278,0.0,0.7,0.0,4,0,9,4,9,2,11,4,1,4,2,0,1,1,1,0,0


In [31]:
for f in train.columns:
    if 'bin' in f:
        print(f)
        print(train[f].value_counts())
        print("===============")

    elif 'cat' in f:
        print(f)
        print(train[f].value_counts().sort_index())
        print("=================")


ps_ind_02_cat
-1       216
 1    431859
 2    123573
 3     28186
 4     11378
Name: ps_ind_02_cat, dtype: int64
ps_ind_04_cat
-1        83
 0    346965
 1    248164
Name: ps_ind_04_cat, dtype: int64
ps_ind_05_cat
-1      5809
 0    528009
 1      8322
 2      4184
 3      8233
 4     18344
 5      1649
 6     20662
Name: ps_ind_05_cat, dtype: int64
ps_ind_06_bin
0    360852
1    234360
Name: ps_ind_06_bin, dtype: int64
ps_ind_07_bin
0    442223
1    152989
Name: ps_ind_07_bin, dtype: int64
ps_ind_08_bin
0    497644
1     97568
Name: ps_ind_08_bin, dtype: int64
ps_ind_09_bin
0    484917
1    110295
Name: ps_ind_09_bin, dtype: int64
ps_ind_10_bin
0    594990
1       222
Name: ps_ind_10_bin, dtype: int64
ps_ind_11_bin
0    594205
1      1007
Name: ps_ind_11_bin, dtype: int64
ps_ind_12_bin
0    589594
1      5618
Name: ps_ind_12_bin, dtype: int64
ps_ind_13_bin
0    594648
1       564
Name: ps_ind_13_bin, dtype: int64
ps_ind_16_bin
1    393330
0    201882
Name: ps_ind_16_bin, dtype: int64


In [17]:
data = []

for f in train.columns:
    
    ## make role
    if f=='target':
        role='target'
    elif f=='id':
        role='id'

    else:
        role='input'

    
    ## make level
    if 'bin' in f or f=='target':
        level = 'binary'

    elif 'cat' in f or f=='id':
        level = 'norminal'

    elif train[f].dtype == int:
        level = 'ordinal'

    elif train[f].dtype == float:
        level = 'interval'

    ## make keep
    if f=='id':
        keep = False
    else:
        keep = True

    ## make dtype

    dtype = train[f].dtype

    data.append({"varname":f,"role":role, "level":level, "keep":keep, "dtype":dtype})

In [19]:
data = pd.DataFrame(data).set_index('varname')
data

,role,level,keep,dtype
varname,,,,
id,id,norminal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,norminal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,norminal,True,int64
ps_ind_05_cat,input,norminal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


In [27]:
pd.DataFrame({"count":data.groupby(['role','level'])['level'].count()})

count
role   level          
id     norminal      1
input  binary       17
       interval     10
       norminal     14
       ordinal      16
target binary        1

In [29]:
data.to_csv(path+'data.csv', index=False)